# Mise en classe

> **Objectifs**
>
> -   Mettre en classe des données



## Lire des données

Comme dans les cours précédents, nous allons travailler sur les cantons suisses [1].

Nous allons commencer par charger ces données depuis un fichier CSV.

Pour rappel, la fonction `read_csv` du module `pandas` permet de lire un fichier CSV
et de le stocker dans un tableau de données. 

Ce tableau contient de nombreuses données. Il est possible de n’afficher
que les premières lignes avec la méthode `head`.

Exécutez le code ci-dessous et prenez connaissance des données :

[1] Source: [OFS - 2019-2020](https://www.bfs.admin.ch/bfs/fr/home/statistiques/statistique-regions/portraits-regionaux-chiffres-cles/cantons.assetdetail.15864478.html)

In [64]:
import pandas as pd

# Lire le fichier stats_cantons.csv et le charger dans 'cantons_data'
cantons_data = pd.read_csv('stats_cantons.csv')

#affiche les 5 1er cantons
cantons_data.head()

## Manipuler des données

Pour rappel, on peut également facilement utiliser des colonnes pour créer de nouvelles colonnes.

Par exemple, on peut créer la colonne pour la densité de population au km2 `densite_pop` qui correspond au nombre d'habitants sur la superficie.

In [65]:
# création de la colonne contenant la densité
# attention ! la popuplation est donnée en milliers d'habitants !
cantons_data["densite_pop"] = (cantons_data["habitants_k"] *1000) / cantons_data["superficie_km2"]

# on n'affiche que les colonnes 'canton' et 'densite_pop'
cantons_data[["canton" ,"densite_pop"]].head(10)

On l'a déjà vu, si on s’intéresse à un canton en particulier, on peut également filtrer les données pour ne garder que la lignes correspondant à ce canton.

On peut également stocker le résultat du filtre dans une nouvelle variable !

Par exemple, pour Neuchâtel, on peut créer un nouveau tableau `neuch_data` qui ne contient que les lignes où la colonne `canton` vaut `NE`.

In [66]:
neuch_data = cantons_data[cantons_data["canton"] == "NE"]

neuch_data.head()

## Mettre en classe

Le module pandas permet de facilement mettre en classe des données numériques :

### amplitude fixe - cut()

amplitude fixe = (val max - val min)

La méthode `cut` permet de créer des classes – selon la méthode de l’amplitude fixe ou des seuils observés – à partir d’une
variable numérique. 

L’argument `bins` (bins = bacs en anglais) peut soit être un chiffre qui indique le nombre de classes, soit une liste qui indique les bornes des
classes. 

Le résultat devra être conservé dans une nouvelle colonne !



In [67]:
# on crée une colonne contenant les 4 "bacs" 
cantons_data["densite_4_cats"] = pd.cut(cantons_data["densite_pop"], bins=4, labels=["faible", "moyenne", "élevée", "très élevée"])

# on affiche tout
print(cantons_data[["canton" ,"densite_pop", "densite_4_cats"]])

On peut aussi définir nous-même les seuils :

In [68]:
# seuils définis manuellement 
cantons_data["densite_4_cats"] = pd.cut(cantons_data["densite_pop"], bins=[0,100,300,1000,5000,50000])

# on affiche tout
print(cantons_data[["canton" ,"densite_pop", "densite_4_cats"]])

### effectifs fixes - qcut()

effectifs fixes = nb de val / nb de classe

La méthode `qcut` permet de créer des classe – selon la méthode des effectifs fixes – à partir d’une variable numérique en précisant le
nombre de classes. 

L’argument `q` permet de préciser le nombre de classes.


In [69]:
# on découpe les données en 4 quantiles
cantons_data["densite_q"] = pd.qcut(cantons_data["densite_pop"], 4, labels=["faible", "moyenne", "élevée", "très élevée"])

# on affiche tout
print(cantons_data[["canton" ,"densite_pop", "densite_q"]])

L’argument `labels` permet de préciser les étiquettes des classes :

In [70]:
cantons_data["densite_4_cats"] = pd.cut(cantons_data["densite_pop"], bins=4, labels=["faible", "moyenne", "élevée", "très élevée"])
print(cantons_data[["canton" ,"densite_pop", "densite_4_cats"]])

In [71]:
cantons_data["densite_q"] = pd.qcut(cantons_data["densite_pop"], 4, labels=["faible", "moyenne", "élevée", "très élevée"])
print(cantons_data[["canton" ,"densite_pop", "densite_q"]])

> **Exercice 1**
>
> -   À partir du tableau `cantons_data`, créez des classes à partir des colonnes suivantes :
>     -   tx_chomage 
>     -   PIB_hab 
>     -   PIB (colonne à créer !)
> - Faites une mise en classe par quartiles (faible, moyen, élevé, très élevé) pour chacune de ces classes.
> -   Affichez le tableau final


> **Exercice 2**
>
> -   Jouez avec le code ci-dessous pour afficher les différentes classes créées en 1


In [72]:
# charge la bibliothèque permettant de faire des cartes
import geopandas

# on crée une colonne contenant les clés pour faire une jointure avec la carte
cantons_data["CodeCH"] = "CH" + cantons_data["canton"]

# chargement de la carte
carte_suisse = geopandas.read_file("ch.shp")

# jointure carte / données
carte_suisse_pop = carte_suisse.merge(
    cantons_data, how="left", left_on="id", right_on="CodeCH"
)

# affichage
carte_suisse_pop.plot(
    column="densite_pop",
    legend=True,
    cmap="OrRd",
    missing_kwds={"color": "grey", "label": "No Data"}
)